In [1]:
import pandas as pd
import numpy as np
import glob
import janitor
import altair as alt
import matplotlib as plt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Data Cleaning

In [3]:
files = glob.glob('data/result_df/*.csv')
dirty_df = pd.concat((pd.read_csv(file, index_col=0)
                for file in files)
              )

dirty_df = dirty_df.clean_names()
dirty_df.drop_duplicates(subset=['listing_code', 'reference_number'], inplace=True)
dirty_df.dropna(subset=['brand', 'model', 'listing_code', 'price', 'title', 'subtitle', 'case_diameter'], inplace=True)
dirty_df.reset_index(drop=True, inplace=True)

print(dirty_df.shape)
dirty_df.head()

(66280, 48)


,listing_code,brand,model,reference_number,movement,case_material,bracelet_material,year_of_production,condition,scope_of_delivery,...,thickness,lug_width,buckle_width,frequency,bracelet_thickness,submariner_kermit_ref_,day_date_ref_,datejust_reference_number,submariner_date_reference,reference
0,IJD7R3,Rolex,Datejust 41,126331 NEW UNWORN 2023 Wimbledon 41mm Jubilee,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOAOQ5,Rolex,Datejust 31,278271,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IJ9RY8,Rolex,Datejust 36,126231,Automatic,Gold/Steel,Gold/Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FDHJM3,Rolex,GMT-Master II,126710BLNR,Automatic,Steel,Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FFF9D3,Rolex,Explorer,124270,Automatic,Steel,Steel,2021,Very good\n(Worn with little to no signs of wear),"Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# clean case_diameter
def is_convertible_to_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

convertible_mask = dirty_df['case_diameter'].str[:2].apply(is_convertible_to_int)

dirty_df = dirty_df[convertible_mask]

dirty_df['case_diameter'] = dirty_df['case_diameter'].str[:2].astype('int')


In [5]:
# add column of whether the price is negotiable
dirty_df.insert(loc=13, column='is_negotiable', value=dirty_df['price'].str.contains('Negotiable', case=False).astype(int))

In [6]:
# keep only CA$ in the `price` column
dirty_df['price'] = dirty_df['price'].str.extract('C\$([0-9,]+)')[0].str.replace(',', '')
dirty_df['price'] = pd.to_numeric(dirty_df['price'], errors='coerce')
dirty_df['price'].fillna(0, inplace=True)
dirty_df['price'] = dirty_df['price'].astype(int)

dirty_df = dirty_df.query('price != 0')

<>:2: DeprecationWarning: invalid escape sequence '\$'
<>:2: DeprecationWarning: invalid escape sequence '\$'
C:\Users\user\AppData\Local\Temp\ipykernel_26808\2926559523.py:2: DeprecationWarning: invalid escape sequence '\$'
  dirty_df['price'] = dirty_df['price'].str.extract('C\$([0-9,]+)')[0].str.replace(',', '')


In [7]:
# add column of whether the year of production is approximated
dirty_df.insert(loc=8, column='year_is_approximated', value=dirty_df['year_of_production'].str.contains('Approximation', case=False).astype(int))

# Clean year of production
dirty_df['year_of_production'] = dirty_df['year_of_production'].apply(lambda x: x[:4] if x != 'Unknown' else x)
dirty_df['year_of_production'] = dirty_df['year_of_production'].replace('Unknown', np.nan)
dirty_df['year_of_production'] = pd.to_numeric(dirty_df['year_of_production'], errors='coerce')

In [8]:
# simplify the location to country only
dirty_df['country'] = dirty_df['location'].str.split(',').str[0]

Save the cleaned data locally

In [9]:
rolex_df = dirty_df.copy()
rolex_df.to_csv('data/rolex_df.csv')